In [1]:
# %pip install -r requirements.txt 

In [2]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import math
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:


# Initialize Pygame
pygame.init()
display = pygame.display.list_modes()[0]  # Get the current screen resolution
pygame.display.set_mode(display, DOUBLEBUF | OPENGL | FULLSCREEN)
pygame.display.set_caption("Solar System Simulation")
pygame.mouse.set_visible(False)
pygame.mixer.init()

try:
    pygame.mixer.music.load("space-rumble-29970.mp3")
    pygame.mixer.music.play(-1)
except:
    print("Could not load music.")
    
gluPerspective(45, (display[0] / display[1]), 0.1, 100.0)
glTranslatef(0.0, 0.0, -40)

# Sun and planet data
planets = [
    {"name": "Sun", "radius": 2.0, "distance": 0, "speed": 0, "color": (1, 1, 0)},
    {"name": "Mercury", "radius": 0.2, "distance": 4, "speed": 0.03, "color": (0.5, 0.5, 0.5)},
    {"name": "Venus", "radius": 0.3, "distance": 5.5, "speed": 0.025, "color": (1, 0.8, 0.2)},
    {"name": "Earth", "radius": 0.5, "distance": 7, "speed": 0.02, "color": (0, 0.5, 1)},
    {"name": "Mars", "radius": 0.4, "distance": 9, "speed": 0.018, "color": (1, 0.3, 0)},
    {"name": "Jupiter", "radius": 1.0, "distance": 12, "speed": 0.012, "color": (1, 0.6, 0.2)},
    {"name": "Saturn", "radius": 0.9, "distance": 16, "speed": 0.009, "color": (1, 1, 0.5)},
    {"name": "Uranus", "radius": 0.7, "distance": 20, "speed": 0.006, "color": (0.5, 1, 1)},
    {"name": "Neptune", "radius": 0.7, "distance": 24, "speed": 0.004, "color": (0.3, 0.5, 1)},
]




In [4]:

# Draw a simple sphere (uses gluSphere)
def draw_sphere(radius, color):
    glColor3fv(color)
    quadric = gluNewQuadric()
    gluSphere(quadric, radius, 32, 16)
    gluDeleteQuadric(quadric)

# Planet angle state
angles = [0 for _ in planets]
class Comet:
    def __init__(self):
        self.reset()

    def reset(self):
        self.x = random.uniform(-60, 60)
        self.y = random.uniform(-30, 30)
        self.z = random.uniform(-100, -70)
        self.dx = random.uniform(0.1, 0.4)
        self.dy = random.uniform(-0.1, 0.1)
        self.dz = random.uniform(0.1, 0.4)

    def update(self):
        self.x += self.dx
        self.y += self.dy
        self.z += self.dz
        if self.z > 0 or abs(self.x) > 70 or abs(self.y) > 40:
            self.reset()

    def draw(self):
        glPushMatrix()
        glTranslatef(self.x, self.y, self.z)
        draw_sphere(0.2, (1, 1, 1))  # white comet
        glPopMatrix()

comets = [Comet() for _ in range(3)]

In [5]:
num_stars = 200
stars = [
    (
        random.uniform(-50, 50),
        random.uniform(-50, 50),
        random.uniform(-90, -40)
    )
    for _ in range(num_stars)
]


In [6]:
# OpenGL error checking
def check_opengl_error():
    error = glGetError()
    if error != GL_NO_ERROR:
        print("OpenGL error:", gluErrorString(error))


In [ ]:
clock = pygame.time.Clock()
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == KEYDOWN and event.key == pygame.K_ESCAPE):
            pygame.quit()
            quit()
        
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glEnable(GL_DEPTH_TEST)

    # Move stars slowly for twinkling effect
    for i in range(len(stars)):
        x, y, z = stars[i]
        z += 0.05
        if z > -40:
            z = random.uniform(-90, -70)
        stars[i] = (x, y, z)

    # Draw background stars
    glPointSize(1)
    glBegin(GL_POINTS)
    glColor3f(1, 1, 1)
    for star in stars:
        glVertex3f(*star)
    glEnd()

    # Draw Sun and planets
    for i, p in enumerate(planets):
        glPushMatrix()

        if i != 0:
            angles[i] += p["speed"]
            x = math.cos(angles[i]) * p["distance"]
            z = math.sin(angles[i]) * p["distance"]
            glTranslatef(x, 0, z)
        draw_sphere(p["radius"], p["color"])

        glPopMatrix()

    for comet in comets:
        comet.update()
        comet.draw()
    check_opengl_error()
    pygame.display.flip()
    clock.tick(60)

GLError: GLError(
	err = 1282,
	description = b'invalid operation',
	baseOperation = glClear,
	cArguments = (16640,)
)

: 